In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
from PIL import Image
import seaborn as sb
from statsmodels.tsa.seasonal import STL
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import urllib
#------------------
sys.path.append("..")
from scripts.dataset import TimeSeriesDataset
from scripts.models import MLR, MLP
from scripts.resnet_family import resnet20_cifar
from scripts.utils_cm import compute_cm

## Generate a synthetic time-series data

In [ ]:
total_days = 31
trend_slope = 15
noise_coef = 2
noise_coef_outlier = 0 * noise_coef

x_period = np.linspace(0, 2*np.pi, 288)
signal = []
for _ in range(total_days):
    y = 10 * np.sin(4 * x_period) * np.exp(-0.5 * x_period)
    y[-100:] = 0
    n = noise_coef * np.random.randn(len(x_period))
    if np.random.choice(7) == 0:
        n += noise_coef_outlier * np.random.randn(len(x_period))
    y += n
    signal.append(y)
signal = np.hstack(signal)

trend = trend_slope * np.linspace(0, 1, len(signal))
signal += trend
x = np.arange(len(signal))

regressor = trend + np.cos(.0005 * x) + 0.1 * np.random.randn(len(trend))

plt.plot(signal)

In [ ]:
stl = STL(signal[:-288], period=288, seasonal=7)
stl_result = stl.fit()

In [ ]:
stl_result.plot()

In [ ]:
seasonal = stl_result.seasonal
trend = stl_result.trend
signal_train_detrend = signal[:-288] - trend
plt.plot(signal_train_detrend)

In [ ]:
train_labels = signal_train_detrend
train_data = np.hstack([seasonal.reshape(-1, 1)])
test_labels = signal[-288:]
test_data = np.hstack([seasonal[-288:].reshape(-1, 1)])

test_trend = ((trend[-1] - trend[-288]) + trend[-288:])

### Add external regressors to the feature set

In [ ]:
# train_data = np.hstack([train_data, regressor[:-288].reshape(-1, 1)])
# test_data = np.hstack([test_data, regressor[-288:].reshape(-1, 1)])

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train_loader = DataLoader(TimeSeriesDataset(train_data, train_labels), batch_size=16, shuffle=True)
test_loader = DataLoader(TimeSeriesDataset(test_data, test_labels), batch_size=16, shuffle=False)

## [Training Model and Configuration](#Training-Model-and-Configuration)

In [ ]:
hidden_layers = [100, 100, 100, 100]
dropout = 0.5
lr=0.001
tau = 1
wd = 0.005 #  0.01**2 * (1 - dropout) / (2. * len(train_loader) * tau)

model = MLR(input_size=train_data.shape[1], nclasses=1, hidden_layers=hidden_layers, dropout=dropout)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
model.net

In [ ]:
loss_vec = []
reset_loss_every = 300
n_epochs = 20
device = "cpu"
model.to(device)
model.train()

for epoch in range(n_epochs):
    for it, batch in enumerate(train_loader):
        optimizer.zero_grad()
        data, target = batch
        output = model(data.float().to(device))
        loss = F.mse_loss(output, target.view(-1, 1).to(device))
        loss_vec.extend([loss.item()])
        loss.backward()
        optimizer.step()
        
        if (it % reset_loss_every) == 0:
            print(f"epoch: {epoch}, it: {it}, average_loss: {np.mean(loss_vec)}")
            loss_vec = []


In [ ]:
model.to(device)
model.train()

ensemble_size = 1000

loss_vec = []
forecast = []
gt = []
for it, batch in enumerate(test_loader):
    data, target = batch
    output = []
    for ensemble_it in range(ensemble_size):
        output.append(model(data.float().to(device)).data.numpy())
    output = np.hstack(output)
    forecast.append(output)
    gt.append(target.view(-1).data.numpy())
    
forecast = np.vstack(forecast)
forecast += test_trend[:, np.newaxis]
predictive_mean = forecast.mean(1)
predictive_std = forecast.std(1)
gt = np.hstack(gt)
mse_loss = ((gt - predictive_mean)**2).mean()
print(f"Test mse_loss = {mse_loss}")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(predictive_mean, label="Forecast")
axes[0].plot(gt, label="Ground Truth")
axes[0].legend()
axes[1].plot(predictive_mean, color='r', label="Forecast")
axes[1].fill_between(np.arange(288), predictive_mean-2*predictive_std, predictive_mean+2*predictive_std, label='prediction interval', color='g', alpha=0.9);
axes[1].legend();

Repeat the above training+evaluation experiment for two different values of DropourRate={0, 0.5} in the [Training-Model-and-Configuration](http://localhost:8888/notebooks/uncertainty_regression.ipynb#Training-Model-and-Configuration) cell (above) and see how that single change introduces the prediction interval.